# Generate plots of the trajectory on the loss landscape
Given:
- ckpt_dir: path to a ckpt directory that contains the ckpted model states for 200 epochs of training
- 

Steps:
1. Compute the trajectories during optimization steps (optimization of the learning objective function)

Inputs: 
- a folder of checkpoints (-s argument), 
- precomputed gradient's directions (npz file)

Effects:
- creates the `trajectory folder` specified by the  "-r (ie. trajectory) folder", and
- saves the projection file in that directory

Command: 
```python
python compute_trajectory.py \
-s '../results/hw1-exp1/resnet20/run_20220212-184545/ckpts/' \
--direction_file "../results/hw1-exp1/resnet20/run_20220212-184545/fds/buffer.npy.npz" \
-r "../results/hw1-exp1/resnet20/run_20220212-184545/trajectories" \
--projection_file fd_dir_proj.npz --model resnet20 
```

Stdout:
```shell
#output looks like following:
2022-02-13 14:57:01,098 using resnet20 with 269722 parameters
2022-02-13 14:57:01,104 Found 201 models
2022-02-13 14:57:18,201 Dot product is 1.6763806343078613e-08
2022-02-13 14:57:18,882 Saving results
2022-02-13 14:57:18,925 xrange: -0.18108321726322174, 0.3358021676540375
2022-02-13 14:57:18,930 yrange: -0.11699468642473221, 0.3524200916290283
```

2.  Compute loss landscapes of final models
This step creates a new folder `loss_surface` inside the `run-<run_id>` folder, and
saves the file as argument of `--surface_file` 

Caution: this takes a long time! (maybe a bug in the provided code; doesn't seem like
using the GPU properly)

Command:
```python
nohup \
python compute_loss_surface.py \
--result_folder results/resnet20_skip_bn_bias_remove_skip_connections/loss_surface/  \
-s "../results/hw1-exp1/resnet20/run_20220212-184545/ckpts/200_model.pt"  \
--batch_size 1000  \
--model resnet20  \
--direction_file "../results/hw1-exp1/resnet20/run_20220212-184545/fds/buffer.npy.npz" \
--surface_file fd_dir_loss_surface.npz \
--gpu_id 0 \
--xcoords 51:-10:40 --ycoords 51:-10:40  > log-compute-loss-surface.txt&             
```

3. Plot the results 
Inputs:
- trajectory file or surface file or both 

Command:
- Use both trajectory and surface files

```python
python plot.py --result_folder figures/resnet56/ \
--trajectory_file r"../results/hw1-exp1/resnet20/run_20220212-184545/trajectories/fd_dir_proj.npz"  \
--surface_file "../results/hw1-exp1/resnet20/run_20220212-184545/loss_surface/fd_dir_loss_surface.npz" \
--plot_prefix resnet20_fd_dir
```

- Use only the trajectory file

```python
python plot.py --result_folder "../results/hw1-exp1/resnet20/run_20220212-184545/plots" \
--trajectory_file "../results/hw1-exp1/resnet20/run_20220212-184545/trajectories/fd_dir_proj.npz"  \
--plot_prefix resnet20_fd_dir_traj
```


In [ ]:
../results/hw1-exp1/resnet20/run_20220212-182238

In [1]:
from pathlib import Path

### Step 1: compute trajectorties in the loss lanscape

In [301]:
exp_name = 'hw1-exp1' #'hw1-exp6-run1'
model_name = 'resnet20'
run_id = 'run_20220212-182238'
use_skip = True

# for fcnets
n_hidden = 128
act_fn = 'relu' #'relu', 'leaky', 'softplus'
use_bn = False


exp_dir = Path(f'../results/{exp_name}/{model_name}/{run_id}')
ckpt_dir =  exp_dir/'ckpts'

# compute trajectory
direction_fp = exp_dir/'fds/buffer.npy.npz'
traj_dir = exp_dir/'trajectories'
proj_type = 'fd'
proj_fn = f'{proj_type}_dir_proj.npz'

# plot trajectory and/or loss contour lines
plot_dir = exp_dir/'plots'
traj_fp = traj_dir/f'{proj_fn}'
# plot_prefix = f"{model_name}_{proj_type}_dir_traj"
plot_prefix = f"{model_name}_{proj_type}" #both traj and surface

# compute loss values along the trajectory
loss_dir = exp_dir/'loss_surface'
final_model_fp = ckpt_dir / '200_model.pt'
compute_loss_bs = 514
loss_surface_fn = f'{proj_type}_dir_loss_surface.npz'
loss_surface_fp = loss_dir / loss_surface_fn

# gpu_id = 0
# gpu_id = 1
# gpu_id = 2
gpu_id = 3


x_min, x_max = -0.35, 0.35
y_min, y_max = -0.35, 0.35




In [302]:
compute_traj_cmd = f'''
python compute_trajectory.py \
-s '{ckpt_dir}' \
--direction_file "{direction_fp}" \
-r "{traj_dir}" \
--projection_file "{proj_fn}" \
--model {model_name}  '''

if 'resnet' in model_name and not use_skip:
    compute_traj_cmd += '--remove_skip_connections  '
    
if 'fcnet' in model_name:
    compute_traj_cmd += f'--n_hidden {n_hidden} --act {act_fn}  '
    if use_bn: 
        compute_traj_cmd += '--use_bn '
    
print(compute_traj_cmd)


python compute_trajectory.py -s '../results/hw1-exp1/resnet20/run_20220212-182238/ckpts' --direction_file "../results/hw1-exp1/resnet20/run_20220212-182238/fds/buffer.npy.npz" -r "../results/hw1-exp1/resnet20/run_20220212-182238/trajectories" --projection_file "fd_dir_proj.npz" --model resnet20  


### Step 2: Compute loss surface
Compute values of loss function at each model checkpoint state (as we move
in the directions (on the domain of the loss function, i.e. the parameter space)
as saved in the `--direction_file` (which is saved as npz file named `{traj_fp}`

In [303]:
compute_loss_surface_cmd=f'''
nohup python compute_loss_surface.py \
-s "{final_model_fp}"  \
--model {model_name}  \
--direction_file "{direction_fp}" \
--batch_size {compute_loss_bs}  \
--result_folder "{loss_dir}"  \
--surface_file "{loss_surface_fn}"  \
--gpu_id {gpu_id}  \
--xcoords 51:{x_min}:{x_max} --ycoords 51:{y_min}:{y_max} \
'''


if 'resnet' in model_name and not use_skip:
    compute_loss_surface_cmd += '--remove_skip_connections  '
    
if 'fcnet' in model_name:
    compute_loss_surface_cmd += f'--n_hidden {n_hidden} --act {act_fn}  '
    if use_bn: 
        compute_loss_surface_cmd += '--use_bn '
    
compute_loss_surface_cmd += '  &'
print(compute_loss_surface_cmd)


nohup python compute_loss_surface.py -s "../results/hw1-exp1/resnet20/run_20220212-182238/ckpts/200_model.pt"  --model resnet20  --direction_file "../results/hw1-exp1/resnet20/run_20220212-182238/fds/buffer.npy.npz" --batch_size 514  --result_folder "../results/hw1-exp1/resnet20/run_20220212-182238/loss_surface"  --surface_file "fd_dir_loss_surface.npz"  --gpu_id 3  --xcoords 51:-0.35:0.35 --ycoords 51:-0.35:0.35   &


### Step 3: Plot the trajectory and loss surface

In [304]:
plot_traj_cmd = f'''
python plot.py --result_folder "{plot_dir}" \
--trajectory_file "{traj_fp}"  \
--surface_file "{loss_surface_fp}"  \
--plot_prefix "{plot_prefix}" 
'''

print(plot_traj_cmd)


python plot.py --result_folder "../results/hw1-exp1/resnet20/run_20220212-182238/plots" --trajectory_file "../results/hw1-exp1/resnet20/run_20220212-182238/trajectories/fd_dir_proj.npz"  --surface_file "../results/hw1-exp1/resnet20/run_20220212-182238/loss_surface/fd_dir_loss_surface.npz"  --plot_prefix "resnet20_fd" 

